In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/utc_working/medes/')

import os
os.chdir('/content/drive/My Drive/utc_working/medes/')


Mounted at /content/drive


In [ ]:
!ls
!pip install -r requirements.txt

250_final_ml100k_user_context_criteria.csv  ml-1m
427_final_ml1m_user_context_criteria.csv    ml1m_statistics_summary.csv
427_ml1m_statistics_summary.csv		    ml_processing.py
575_final_ml1m_user_context_criteria.csv    outputs
575_grouped_ml1m_context_criteria.csv	    __pycache__
575_ml1m_statistics_summary.csv		    requirements.txt
final_ml100k_user_context_criteria.csv	    run.ipynb
final_ml1m_user_context_criteria.csv	    run_movielen_100k.py
grouped_ml1m_context_criteria.csv	    run_mvdb.py
imha_model.py				    run_mv.sh
ITM-Rec					    run.sh
main.py					    visualization.py
mha_model.py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install torchinfo

In [ ]:
!lscpu
!cat /proc/meminfo | grep Mem
!nvidia-smi --query-gpu=name --format=csv,noheader


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

# **Model Summary**

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import torch
from torchinfo import summary
import torch.nn as nn
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models.basemodel import BaseModel
from deepctr_torch.layers import PredictionLayer
from deepctr_torch.models import (
    AFM,
    AutoInt,
    CCPM,
    DCN,
    DeepFM,
    DIFM,
    FiBiNET,
    NFM,
    ONN,
    PNN,
    WDL,
    xDeepFM,
    IFM,
    MLR
)
from mha_model import *
# Model Summary Utility unchanged below


# ----------------------------
# Step 1: Load and Merge Data
# ----------------------------
def run(dense_features, sparse_features):
    # ----------------------------
  # Step 1: Load and Merge Data
  # ----------------------------

  # Load Data
  users = pd.read_csv("./ITM-Rec/users.csv")
  items = pd.read_csv("./ITM-Rec/items.csv")
  df_ratings = pd.read_csv("./ITM-Rec/ratings.csv")

  # Load group data
  group_df = pd.read_csv("./ITM-Rec/group.csv")
  group_size_df = pd.read_csv("./ITM-Rec/group_size.csv")
  group_ratings = pd.read_csv("./ITM-Rec/group_ratings.csv")

  # Rename columns for clarity
  group_ratings.columns = ['GroupID', "Item", 'GroupRating', 'GroupApp', 'GroupData', 'GroupEase', 'GroupClass', 'GroupSemester', 'GroupLockdown']

  # Initial shapes
  print("Shape of df_ratings before merge:", df_ratings.shape)
  print("Shape of group_df:", group_df.shape)
  print("Shape of group_size_df:", group_size_df.shape)
  print("Shape of group_ratings:", group_ratings.shape)

  # Select relevant columns from individual ratings
  df_ratings = df_ratings[["UserID", "Item", "Class", "Semester", "Lockdown", "App", "Data", "Ease", "Rating"]]

  # Merge with group information
  df_ratings = pd.merge(df_ratings, group_df, on='UserID', how='left')

  # Merge with group size
  df_ratings = pd.merge(df_ratings, group_size_df, on='GroupID', how='left')

  # Merge with group ratings
  df_ratings = pd.merge(df_ratings, group_ratings, on=['GroupID', 'Item'], how='left')

  # Fill missing GroupID with UserID (for users not in any group, treat them as single-member groups)
  df_ratings['GroupID'] = df_ratings['GroupID'].fillna(df_ratings['UserID'])


  # Fill missing group sizes with 1 (single-member groups)
  df_ratings['Size'] = df_ratings['Size'].fillna(1)

  # Fill missing group ratings with individual ratings
  for col in ['Rating', 'App', 'Data', 'Ease', 'Class', 'Semester', 'Lockdown']:
      group_col = f'Group{col}'
      df_ratings[group_col] = df_ratings[group_col].fillna(df_ratings[col])

  print("df_rating: ", df_ratings.head())

  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])
  df_ratings = df_ratings.dropna()
  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])


  print("df_rating", df_ratings.head())
  # Check for any remaining missing values
  print("\nMissing values after merging:")
  print(df_ratings.isnull().sum())

  # ----------------------------
  # Step 2: Feature Engineering
  # ----------------------------

  # Initialize LabelEncoders
  label_encoders = {}

  # Initialize and fit LabelEncoders
  print("sparse_features", sparse_features)
  for feat in sparse_features:
      print("feat: ", feat)
      le = LabelEncoder()
      df_ratings[feat] = le.fit_transform(df_ratings[feat].astype(str))
      label_encoders[feat] = le


  # Initialize MinMaxScaler
  scaler = MinMaxScaler()

  # Fit and transform the dense features
  df_ratings[dense_features] = scaler.fit_transform(df_ratings[dense_features])

  # Define target variables
  #target = ['GroupRating', 'GroupApp', 'GroupData', 'GroupEase']
  target = ['GroupRating']
  # ----------------------------
  # Step 3: Prepare Data for Modeling
  # ----------------------------

  # Split the data
  train_data, test_data = train_test_split(df_ratings, test_size=0.2, random_state=42)

  # Define feature columns
  # Define feature columns
  sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df_ratings[feat].nunique(), embedding_dim=4) for feat in sparse_features]
  dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

  feature_columns = sparse_feature_columns + dense_feature_columns

  print("feature columns: ", feature_columns)
  # Get feature names
  feature_names = get_feature_names(feature_columns)

  # Generate input data for model
  train_model_input = {name: train_data[name].values for name in feature_names}
  test_model_input = {name: test_data[name].values for name in feature_names}

  # Ensure data types are correct
  for name in feature_names:
      if name in sparse_features:
          train_model_input[name] = train_model_input[name].astype(int)
          test_model_input[name] = test_model_input[name].astype(int)
      else:
          train_model_input[name] = train_model_input[name].astype(float)
          test_model_input[name] = test_model_input[name].astype(float)



  # Update the models dictionary
  linear_feature_columns = feature_columns
  dnn_feature_columns =  feature_columns
  num_heads = 4
  dnn_hidden_units = (256, 128, 64, 32, 16, 8)
  l2_reg_dnn = 1e-5
  dnn_dropout = 0.42

  model = MultiHeadAttentionModel(
      linear_feature_columns=linear_feature_columns,
      dnn_feature_columns=dnn_feature_columns,
      num_heads=num_heads,
      dnn_hidden_units=dnn_hidden_units,
      dnn_dropout=dnn_dropout,
      l2_reg_dnn=l2_reg_dnn,
      device="cuda" if torch.cuda.is_available() else "cpu"
  )

  print(model)
  print(summary(model))
dense_features = ['App', 'Data', 'Ease',
                    'GroupApp', 'GroupData', 'GroupEase']
    ## Context = ['Class', 'Semester', 'Lockdown']
    # Define categorical features
sparse_features = ['UserID', 'Item', 'Class', 'Semester', 'Lockdown',
                    'GroupID', 'GroupClass', 'GroupSemester', 'GroupLockdown']
run(dense_features, sparse_features)

Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         DA          Fall   
3   2.0          5.0       4.0        4.0        4.0         D

# **MCGRS-MC: Running Model Training with all context and criteria**

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import torch
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models.basemodel import BaseModel
from deepctr_torch.layers import PredictionLayer
from deepctr_torch.models import (
    AFM,
    AutoInt,
    CCPM,
    DCN,
    DeepFM,
    DIFM,
    FiBiNET,
    NFM,
    ONN,
    PNN,
    WDL,
    xDeepFM,
    IFM,
    MLR
)
from mha_model import *

# ----------------------------
# Step 1: Load and Merge Data
# ----------------------------
def run(dense_features, sparse_features):
    # ----------------------------
  # Step 1: Load and Merge Data
  # ----------------------------

  # Load Data
  users = pd.read_csv("./ITM-Rec/users.csv")
  items = pd.read_csv("./ITM-Rec/items.csv")
  df_ratings = pd.read_csv("./ITM-Rec/ratings.csv")

  # Load group data
  group_df = pd.read_csv("./ITM-Rec/group.csv")
  group_size_df = pd.read_csv("./ITM-Rec/group_size.csv")
  group_ratings = pd.read_csv("./ITM-Rec/group_ratings.csv")

  # Rename columns for clarity
  group_ratings.columns = ['GroupID', "Item", 'GroupRating', 'GroupApp', 'GroupData', 'GroupEase', 'GroupClass', 'GroupSemester', 'GroupLockdown']

  # Initial shapes
  print("Shape of df_ratings before merge:", df_ratings.shape)
  print("Shape of group_df:", group_df.shape)
  print("Shape of group_size_df:", group_size_df.shape)
  print("Shape of group_ratings:", group_ratings.shape)

  # Select relevant columns from individual ratings
  df_ratings = df_ratings[["UserID", "Item", "Class", "Semester", "Lockdown", "App", "Data", "Ease", "Rating"]]

  # Merge with group information
  df_ratings = pd.merge(df_ratings, group_df, on='UserID', how='left')

  # Merge with group size
  df_ratings = pd.merge(df_ratings, group_size_df, on='GroupID', how='left')

  # Merge with group ratings
  df_ratings = pd.merge(df_ratings, group_ratings, on=['GroupID', 'Item'], how='left')

  # Fill missing GroupID with UserID (for users not in any group, treat them as single-member groups)
  df_ratings['GroupID'] = df_ratings['GroupID'].fillna(df_ratings['UserID'])


  # Fill missing group sizes with 1 (single-member groups)
  df_ratings['Size'] = df_ratings['Size'].fillna(1)

  # Fill missing group ratings with individual ratings
  for col in ['Rating', 'App', 'Data', 'Ease', 'Class', 'Semester', 'Lockdown']:
      group_col = f'Group{col}'
      df_ratings[group_col] = df_ratings[group_col].fillna(df_ratings[col])

  print("df_rating: ", df_ratings.head())

  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])
  df_ratings = df_ratings.dropna()
  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])


  print("df_rating", df_ratings.head())
  # Check for any remaining missing values
  print("\nMissing values after merging:")
  print(df_ratings.isnull().sum())

  # ----------------------------
  # Step 2: Feature Engineering
  # ----------------------------

  # Initialize LabelEncoders
  label_encoders = {}

  # Initialize and fit LabelEncoders
  print("sparse_features", sparse_features)
  for feat in sparse_features:
      print("feat: ", feat)
      le = LabelEncoder()
      df_ratings[feat] = le.fit_transform(df_ratings[feat].astype(str))
      label_encoders[feat] = le


  # Initialize MinMaxScaler
  scaler = MinMaxScaler()

  # Fit and transform the dense features
  df_ratings[dense_features] = scaler.fit_transform(df_ratings[dense_features])

  # Define target variables
  #target = ['GroupRating', 'GroupApp', 'GroupData', 'GroupEase']
  target = ['GroupRating']
  # ----------------------------
  # Step 3: Prepare Data for Modeling
  # ----------------------------

  # Split the data
  train_data, test_data = train_test_split(df_ratings, test_size=0.2, random_state=42)

  # Define feature columns
  # Define feature columns
  sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df_ratings[feat].nunique(), embedding_dim=4) for feat in sparse_features]
  dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

  feature_columns = sparse_feature_columns + dense_feature_columns

  print("feature columns: ", feature_columns)
  # Get feature names
  feature_names = get_feature_names(feature_columns)

  # Generate input data for model
  train_model_input = {name: train_data[name].values for name in feature_names}
  test_model_input = {name: test_data[name].values for name in feature_names}

  # Ensure data types are correct
  for name in feature_names:
      if name in sparse_features:
          train_model_input[name] = train_model_input[name].astype(int)
          test_model_input[name] = test_model_input[name].astype(int)
      else:
          train_model_input[name] = train_model_input[name].astype(float)
          test_model_input[name] = test_model_input[name].astype(float)



  # Update the models dictionary

  '''

      # AFN (Adaptive Factorization Network)
  '''
  models = {
            "AFM": (AFM, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,  # AFM still uses DenseFeat here
          'dnn_feature_columns': sparse_feature_columns,  # REMOVE DenseFeat here
          'attention_factor': 8,
          'l2_reg_att': 1e-5,
          'use_attention': True,
      }),
      "AutoInt": (AutoInt, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,
          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,  # Keep both for other models
          'att_layer_num': 3,
          'dnn_hidden_units': (256, 128),
      }),
      "CCPM": (CCPM, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,  # Keep DenseFeat in linear features
          'dnn_feature_columns': sparse_feature_columns,  # REMOVE DenseFeat from dnn_feature_columns
          'conv_kernel_width': (6, 5),
          'conv_filters': (4, 4),
          'dnn_hidden_units': (256, 128),
      }),
      "DCN": (DCN, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,
          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,
          'cross_num': 3,
          'dnn_hidden_units': (256, 128),
      }),
      "DeepFM": (DeepFM, {
          'dnn_feature_columns': feature_columns,
      'linear_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "DIFM": (DIFM, {
          'dnn_feature_columns': feature_columns,
          'linear_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "FiBiNET": (FiBiNET, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "NFM": (NFM, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "ONN": (ONN, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "PNN": (PNN, {
          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,  # Only dnn_feature_columns
          'dnn_hidden_units': (256, 128),
          'use_inner': True,
          'use_outter': False,
      }),
      "WDL": (WDL, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "xDeepFM": (xDeepFM, {
          'linear_feature_columns': feature_columns,
        'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
          'cin_layer_size': (128, 128),
      }),
      "IFM": (IFM, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,

          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,
          'dnn_hidden_units': (256, 128),
      }),

      "LS-PLM": (MLR, {
          'region_feature_columns': sparse_feature_columns + dense_feature_columns,
        'base_feature_columns': sparse_feature_columns,  # Example
        'bias_feature_columns': dense_feature_columns,  # Example
      }),


      "MHA": (MultiHeadAttentionModel, {
        'linear_feature_columns': feature_columns,
        'dnn_feature_columns': feature_columns,
        'num_heads': 4,
        'dnn_hidden_units': (256, 128, 64, 32, 16, 8),
        'l2_reg_dnn': 1e-5,
        'dnn_dropout': 0.42,
      })
  }

  def ndcg_at_k(r, k):
    r = np.asarray(r, dtype=float)[:k]
    if r.size == 0:
      return 0.0
    dcg = np.sum((2 ** r - 1) / np.log2(np.arange(2, r.size + 2)))
    ideal_r = np.sort(r)[::-1]
    idcg = np.sum((2 ** ideal_r - 1) / np.log2(np.arange(2, ideal_r.size + 2)))
    return dcg / idcg if idcg != 0 else 0.0


  def precision_at_k(y_true, y_pred, k, relevance_threshold=3.7):  # Add relevance threshold
      idx = np.argsort(y_pred)[::-1][:k]
      return np.sum(np.array(y_true)[idx] >= relevance_threshold) / k  # Apply threshold

  def recall_at_k(y_true, y_pred, k, relevance_threshold=3.7):  # Add relevance threshold
      relevant = np.sum(np.array(y_true) >= relevance_threshold)  # Apply threshold
      idx = np.argsort(y_pred)[::-1][:k]
      return np.sum(np.array(y_true)[idx] >= relevance_threshold) / relevant if relevant > 0 else np.nan  # Handle zero division

  def f1_at_k(precision, recall):
      return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

  # Initialize results dictionary
  results = {}

  # Device configuration
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # Function to measure CPU & memory usage
  def get_resource_usage():
      process = psutil.Process()
      cpu_percent = psutil.cpu_percent(interval=1)  # CPU usage over 1 second
      memory_info = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
      return cpu_percent, memory_info

  # Train and evaluate each model

  for model_name, (model_class, model_params) in models.items():
      print(f"\nTraining and evaluating {model_name}...")

      # Start measuring time and resource usage
      start_time = time.time()
      cpu_before, memory_before = get_resource_usage()

      # Define common parameters
      common_params = {
          'task': 'regression',
          'device': device
      }
      # Update with model-specific parameters
      common_params.update(model_params)

      # Handle models that require special inputs
      if model_name in ['AFM']:
          common_params['use_attention'] = True

      # Define the model
      model = model_class(**common_params)
      model.to(device)

      # Compile the model
      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
      model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

      # Define Early Stopping
      early_stopping = EarlyStopping(monitor='val_mse', patience=5, verbose=1)
      time.sleep(2)  # Simulate some work (replace with model training)
      # Train the model
      history = model.fit(
          train_model_input,
          train_data[target].values,
          batch_size=256,
          epochs=50,
          verbose=2,
          validation_split=0.3,
          callbacks=[early_stopping]
      )

      # Predict on test set
      predictions = model.predict(test_model_input, batch_size=256)

      # Calculate RMSE and MAE
      rmse = np.sqrt(mean_squared_error(test_data[target].values, predictions))
      mae = mean_absolute_error(test_data[target].values, predictions)

      k = 5
      test_df = test_data.copy()
      test_df["prediction"] = predictions

      # Compute NDCG per GroupID
      precisions5, precisions10, recalls, f1s, ndcgs5,ndcgs10 = [], [], [], [], [], []
      for group_id, group_df in test_df.groupby("GroupID"):
          true_ratings = group_df[target[0]].values
          pred_scores = group_df["prediction"].values

          # Sort items by predicted scores
          sorted_idx = np.argsort(pred_scores)[::-1]
          sorted_true = true_ratings[sorted_idx]

          prec5 = precision_at_k(true_ratings, pred_scores, 5)
          prec10 = precision_at_k(true_ratings, pred_scores, 10)


          ndcg5 = ndcg_at_k(sorted_true, 5)
          ndcg10 = ndcg_at_k(sorted_true, 10)


          precisions5.append(prec5)
          precisions10.append(prec10)
          ndcgs5.append(ndcg5)
          ndcgs10.append(ndcg10)

      avg_precision5 = np.mean(precisions5)
      avg_precision10 = np.mean(precisions10)
      avg_ndcg5 = np.mean(ndcgs5)
      avg_ndcg10 = np.mean(ndcgs10)

      # Convert MSE to RMSE and store both RMSE & MAE in history
      history.history['RMSE'] = np.sqrt(history.history['mse'])  # Convert MSE to RMSE
      history.history['val_RMSE'] = np.sqrt(history.history['val_mse'])  # Convert validation MSE to RMSE


      # Measure execution time and resource usage
      end_time = time.time()
      cpu_after, memory_after = get_resource_usage()
      execution_time = end_time - start_time
      cpu_usage = cpu_after - cpu_before
      memory_usage = memory_after - memory_before

      # Store the results
      results[model_name] = {
          'RMSE': rmse,
          'MAE': mae,
          'NDCG@5':avg_ndcg5,
          'NDCG@10':avg_ndcg10,
          'Precision@5': avg_precision5,
          'Precision@10': avg_precision10,
          'Execution Time (s)': execution_time,
          'CPU Usage (%)': cpu_usage,
          'Memory Usage (MB)': memory_usage
      }

      print(f"{model_name} Model RMSE: {rmse:.4f}")
      print(f"{model_name} Model MAE: {mae:.4f}")
      print(f"{model_name} Model NDCG@5: {avg_ndcg5:.4f}")
      print(f"{model_name} Model NDCG@10: {avg_ndcg10:.4f}")
      print(f"{model_name} Model Precision@5: {avg_precision5:.4f}")
      print(f"{model_name} Model Precision@10: {avg_precision10:.4f}")
      print(f"{model_name} Execution Time: {execution_time:.2f} seconds")
      print(f"{model_name} CPU Usage: {cpu_usage:.2f}%")
      print(f"{model_name} Memory Usage: {memory_usage:.2f} MB")

      # Save the history for MAE and RMSE after training each model
      with open(f'./outputs/pickles/{model_name}_imt_history.pkl', 'wb') as file_pi:
          pickle.dump(history.history, file_pi)

  # Display all results
  print("\nFinal Results:")
  for model_name, metrics in results.items():
      #print(f"{model_name} - RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}, "
      #      f"Time: {metrics['Execution Time (s)']:.2f}s, CPU: {metrics['CPU Usage (%)']:.2f}%, "
      #      f"Memory: {metrics['Memory Usage (MB)']:.2f}MB")
      print(f"{model_name} - RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}, ",
        f"NDCG@5: {metrics['NDCG@5']:.4f}, ", f"Precision@5: {metrics['Precision@5']:.4f},",
            f"NDCG@10: {metrics['NDCG@10']:.4f}, ",
            f"Precision@10: {metrics['Precision@10']:.4f}, ",
            f"Time: {metrics['Execution Time (s)']:.2f}s, ",       f"CPU: {metrics['CPU Usage (%)']:.2f}%, Memory: {metrics['Memory Usage (MB)']:.2f}MB")
  #sys.stdout.close()
  #sys.stdout = sys.__stdout__

print("Executing Case 1: Running Model Training with all context and criteria")
    # Define dense features
    #===========================================================
    # test case 1: all context and criteria
    #Criteria ['GroupApp', 'GroupData', 'GroupEase']
dense_features = ['App', 'Data', 'Ease',
                    'GroupApp', 'GroupData', 'GroupEase']
    ## Context = ['Class', 'Semester', 'Lockdown']
    # Define categorical features
sparse_features = ['UserID', 'Item', 'Class', 'Semester', 'Lockdown',
                    'GroupID', 'GroupClass', 'GroupSemester', 'GroupLockdown']
run(dense_features, sparse_features)

Executing Case 1: Running Model Training with all context and criteria
Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         DA         

# **GRS: Running Model Training without all context and criteria**

In [ ]:
print("Executing Case 2: Running Model Training without all context and criteria")
    # Define dense features
    #===========================================================
    # test case 1: all context and criteria
    #Criteria ['GroupApp', 'GroupData', 'GroupEase']
dense_features = ['App', 'Data', 'Ease']
    ## Context = ['Class', 'Semester', 'Lockdown'] 'Class', 'Semester', 'Lockdown',
    # Define categorical features
sparse_features = ['UserID', 'Item', 'GroupID']
run(dense_features, sparse_features)

Executing Case 2: Running Model Training without all context and criteria
Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         DA      

# **MCGRS: Running Prediction with criteria, without context**

In [ ]:
print("Executing Case 3: Running Prediction with criteria, without context")
     #===========================================================
    # test case 1: all context and criteria
    #Criteria ['GroupApp', 'GroupData', 'GroupEase']
dense_features = ['App', 'Data', 'Ease',
                    'GroupApp', 'GroupData', 'GroupEase']
    ## Context = ['Class', 'Semester', 'Lockdown']
    # Define categorical features
sparse_features = ['UserID', 'Item', 'GroupID', 'GroupClass', 'GroupSemester', 'GroupLockdown']
run(dense_features, sparse_features)

Executing Case 3: Running Prediction with criteria, without context
Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         DA          Fa

# **MCGRS-SC-C: Running Prediction with criteria, with single context: class**

In [ ]:
print("Executing Case 5: Running Prediction with criteria, with single context: class")
     #===========================================================
    # test case 1: all context and criteria
    #Criteria ['GroupApp', 'GroupData', 'GroupEase']
dense_features = ['App', 'Data', 'Ease', 'GroupApp', 'GroupData', 'GroupEase']
    ## Context = ['Class', 'Semester', 'Lockdown']
    # Define categorical features
sparse_features = ['UserID', 'Item', 'Class', 'GroupID', 'GroupClass', 'GroupSemester', 'GroupLockdown']
run(dense_features, sparse_features)

Executing Case 5: Running Prediction with criteria, with single context: class
Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         DA 

# **MCGRS-SC-S: Running Prediction with criteria, with single context: semester**

In [ ]:
print("Executing Case 6: Running Prediction with criteria, with single context: semester")
     #===========================================================
    # test case 1: all context and criteria
    #Criteria ['GroupApp', 'GroupData', 'GroupEase']
dense_features = ['App', 'Data', 'Ease', 'GroupApp', 'GroupData', 'GroupEase']
    ## Context = ['Class', 'Semester', 'Lockdown']
    # Define categorical features
sparse_features = ['UserID', 'Item', 'Semester', 'GroupID', 'GroupClass', 'GroupSemester', 'GroupLockdown']
run(dense_features, sparse_features)

Executing Case 6: Running Prediction with criteria, with single context: semester
Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         

# **MCGRS-SC-L: Running Prediction with criteria, with single context: Lockdown**

In [ ]:
print("Executing Case 7: Running Prediction with criteria, with single context: Lockdown")
     #===========================================================
    # test case 1: all context and criteria
    #Criteria ['GroupApp', 'GroupData', 'GroupEase']
dense_features = ['App', 'Data', 'Ease', 'GroupApp', 'GroupData', 'GroupEase']
    ## Context = ['Class', 'Semester', 'Lockdown']
    # Define categorical features
sparse_features = ['UserID', 'Item', 'Lockdown', 'GroupID', 'GroupClass', 'GroupSemester', 'GroupLockdown']
run(dense_features, sparse_features)

Executing Case 7: Running Prediction with criteria, with single context: Lockdown
Shape of df_ratings before merge: (5230, 9)
Shape of group_df: (395, 2)
Shape of group_size_df: (143, 2)
Shape of group_ratings: (1117, 9)
df_rating:     UserID  Item Class Semester Lockdown  App  Data  Ease  Rating  GroupID  \
0    1173    28    DA     Fall      PRE    4   4.0     4       5    124.0   
1    1173    28    DA     Fall      PRE    4   4.0     4       5    144.0   
2    1173    28    DA     Fall      PRE    4   4.0     4       5    199.0   
3    1173    41    DA     Fall      PRE    4   4.0     4       5    124.0   
4    1173    41    DA     Fall      PRE    4   4.0     4       5    144.0   

   Size  GroupRating  GroupApp  GroupData  GroupEase GroupClass GroupSemester  \
0   2.0          5.0       4.0        4.0        4.0         DA          Fall   
1   2.0          5.0       4.0        4.0        4.0         DA          Fall   
2   3.0          5.0       4.0        4.0        4.0         